In [0]:
from pyspark.sql import SparkSession
spark

In [0]:
#1 Data Ingestion & Schema Analysis
# Load CSV using PySpark with schema inference
file_path = "file:/Workspace/Shared/traffic_logs.csv"

df_auto = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("timestampFormat", "yyyy-MM-dd HH:mm")
    .csv(file_path)
)

# Show inferred schema and data
df_auto.printSchema()
df_auto.show()

# Manually define schema and compare
from pyspark.sql.types import *

manual_schema = StructType([
    StructField("LogID", StringType(), True),
    StructField("VehicleID", StringType(), True),
    StructField("EntryPoint", StringType(), True),
    StructField("ExitPoint", StringType(), True),
    StructField("EntryTime", TimestampType(), True),
    StructField("ExitTime", TimestampType(), True),
    StructField("VehicleType", StringType(), True),
    StructField("SpeedKMH", IntegerType(), True),
    StructField("TollPaid", IntegerType(), True),
])

df_manual = spark.read.option("header", "true").schema(manual_schema).csv(file_path)
df_manual.printSchema()
df_manual.show()



root
 |-- LogID: string (nullable = true)
 |-- VehicleID: string (nullable = true)
 |-- EntryPoint: string (nullable = true)
 |-- ExitPoint: string (nullable = true)
 |-- EntryTime: timestamp (nullable = true)
 |-- ExitTime: timestamp (nullable = true)
 |-- VehicleType: string (nullable = true)
 |-- SpeedKMH: integer (nullable = true)
 |-- TollPaid: integer (nullable = true)

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:0

In [0]:
#2 Derived Column Creation
# Add TripDurationMinutes and IsOverspeed columns
from pyspark.sql.functions import col, expr

df_derived = df_manual.withColumn(
    "TripDurationMinutes",
    (expr("unix_timestamp(ExitTime) - unix_timestamp(EntryTime)") / 60).cast("int")
).withColumn(
    "IsOverspeed",
    col("SpeedKMH") > 60
)

df_derived.show()


+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|                 19|      false|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|                 35|      false|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|                 18|      false|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|                 20|       true|
| L005|     V

In [0]:
#3 Vehicle Behavior Aggregations
from pyspark.sql.functions import avg, sum, count, col

# Average speed per VehicleType
df_derived.groupBy("VehicleType").agg(avg("SpeedKMH").alias("AvgSpeed")).show()

# Total toll collected per EntryPoint
df_derived.groupBy("EntryPoint").agg(sum("TollPaid").alias("TotalToll")).show()

# Most used ExitPoint
df_derived.groupBy("ExitPoint").agg(count("*").alias("TripCount")).orderBy(col("TripCount").desc()).show(1)

+-----------+--------+
|VehicleType|AvgSpeed|
+-----------+--------+
|       Bike|    55.0|
|        Car|    70.0|
|      Truck|    45.0|
|        Bus|    40.0|
+-----------+--------+

+----------+---------+
|EntryPoint|TotalToll|
+----------+---------+
|     GateA|       80|
|     GateB|      170|
|     GateC|       50|
+----------+---------+

+---------+---------+
|ExitPoint|TripCount|
+---------+---------+
|    GateD|        2|
+---------+---------+
only showing top 1 row



In [0]:
#4 Window Functions

# Rank vehicles by speed within VehicleType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

w = Window.partitionBy("VehicleType").orderBy(col("SpeedKMH").desc())
df_ranked = df_derived.withColumn("SpeedRank", rank().over(w))
df_ranked.show()

# Find last exit time for each vehicle using lag()
from pyspark.sql.functions import lag

w_vehicle = Window.partitionBy("VehicleID").orderBy("EntryTime")

df_lagged = df_derived.withColumn("LastExitTime", lag("ExitTime").over(w_vehicle))
df_lagged.select("VehicleID", "EntryTime", "ExitTime", "LastExitTime").show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|                 18|      false|        1|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|                 35|      false|        1|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|                 20|       true|        1|
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|    

In [0]:
#5 Session Segmentation
# Calculate idle time between trips
from pyspark.sql.functions import unix_timestamp, round

df_sessions = df_lagged.withColumn(
    "IdleTimeMinutes",
    round((unix_timestamp("EntryTime") - unix_timestamp("LastExitTime")) / 60, 2)
)

df_sessions.select("VehicleID", "EntryTime", "ExitTime", "LastExitTime", "IdleTimeMinutes").show()

+---------+-------------------+-------------------+------------+---------------+
|VehicleID|          EntryTime|           ExitTime|LastExitTime|IdleTimeMinutes|
+---------+-------------------+-------------------+------------+---------------+
|     V001|2024-05-01 08:01:00|2024-05-01 08:20:00|        NULL|           NULL|
|     V002|2024-05-01 08:10:00|2024-05-01 08:45:00|        NULL|           NULL|
|     V003|2024-05-01 09:00:00|2024-05-01 09:18:00|        NULL|           NULL|
|     V004|2024-05-01 09:15:00|2024-05-01 09:35:00|        NULL|           NULL|
|     V005|2024-05-01 10:05:00|2024-05-01 10:40:00|        NULL|           NULL|
+---------+-------------------+-------------------+------------+---------------+



In [0]:
#6 Anomaly Detection
# Speed > 70 and TripDuration < 10
df_anomaly1 = df_derived.filter((col("SpeedKMH") > 70) & (col("TripDurationMinutes") < 10))
df_anomaly1.show()

# Paid less toll for longer trips
df_anomaly2 = df_derived.filter((col("TripDurationMinutes") > 30) & (col("TollPaid") < 50))
df_anomaly2.show()

# Suspicious backtracking (ExitPoint < EntryPoint)
df_anomaly3 = df_derived.filter(col("ExitPoint") < col("EntryPoint"))
df_anomaly3.show()

+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime|ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+

+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime|ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+

+-----+---------+----------+---------+

In [0]:
#7 Join with Metadata
# Create vehicle registry and join
from pyspark.sql import Row

vehicle_data = [
    Row(VehicleID="V001", OwnerName="Anil", Model="Hyundai i20", RegisteredCity="Delhi"),
    Row(VehicleID="V002", OwnerName="Rakesh", Model="Tata Truck", RegisteredCity="Chennai"),
    Row(VehicleID="V003", OwnerName="Sana", Model="Yamaha R15", RegisteredCity="Mumbai"),
    Row(VehicleID="V004", OwnerName="Neha", Model="Honda City", RegisteredCity="Bangalore"),
    Row(VehicleID="V005", OwnerName="Zoya", Model="Volvo Bus", RegisteredCity="Pune"),
]

df_registry = spark.createDataFrame(vehicle_data)

# Join and group trips by RegisteredCity
df_joined = df_derived.join(df_registry, on="VehicleID", how="left")
df_joined.groupBy("RegisteredCity").count().show()

+--------------+-----+
|RegisteredCity|count|
+--------------+-----+
|     Bangalore|    1|
|       Chennai|    1|
|        Mumbai|    1|
|          Pune|    1|
|         Delhi|    1|
+--------------+-----+



In [0]:
#8 Delta Lake Features
# Save as Delta Table
df_derived.write.format("delta").mode("overwrite").save("/Workspace/Shared/traffic_delta")

# MERGE INTO: Update toll for bikes
from delta.tables import DeltaTable

traffic_delta = DeltaTable.forPath(spark, "/Workspace/Shared/traffic_delta")

traffic_delta.alias("old").merge(
    df_derived.filter(col("VehicleType") == "Bike").withColumn("TollPaid", expr("TollPaid + 10")).alias("new"),
    "old.LogID = new.LogID"
).whenMatchedUpdate(set={"TollPaid": "new.TollPaid"}).execute()

# Delete long trips > 60 mins
traffic_delta.delete("TripDurationMinutes > 60")

# Delta versioning and history
spark.sql("DESCRIBE HISTORY delta.`/Workspace/Shared/traffic_delta`").show()

# Read version 0 of delta table
df_v0 = spark.read.format("delta").option("versionAsOf", 0).load("/Workspace/Shared/traffic_delta")
df_v0.show()


+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     11|2025-06-19 05:53:39|8778822765517627|azuser3551_mml.lo...| OPTIMIZE|{predicate -> [],...|NULL|{3072619134359810}|0611-043435-vg20yowf|          9|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|     10|2025-06-19 05:53:37

In [0]:
#9 Advanced Conditions

# Classify trip type
from pyspark.sql.functions import when

df_trip_types = df_derived.withColumn(
    "TripType",
    when(col("TripDurationMinutes") < 15, "Short")
    .when(col("TripDurationMinutes").between(15, 30), "Medium")
    .otherwise("Long")
)

df_trip_types.show()

# Flag vehicles with > 3 trips/day
from pyspark.sql.functions import to_date

df_flagged = df_trip_types.withColumn("TripDate", to_date("EntryTime"))

df_trip_count = df_flagged.groupBy("VehicleID", "TripDate").count().filter("count > 3")
df_trip_count.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|TripType|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+--------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|                 19|      false|  Medium|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|                 35|      false|    Long|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|                 18|      false|  Medium|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80| 

In [0]:

#10 Export & Reporting

# Save to Parquet partitioned by VehicleType
df_derived.write.mode("overwrite").partitionBy("VehicleType").parquet("/Workspace/Shared/traffic_parquet")

# Save to CSV for dashboard
df_derived.write.mode("overwrite").option("header", "true").csv("/Workspace/Shared/traffic_dashboard_csv")

# Summary SQL View: total toll by VehicleType & ExitPoint
df_derived.createOrReplaceTempView("traffic_summary")

spark.sql("""
    SELECT VehicleType, ExitPoint, SUM(TollPaid) as TotalToll
    FROM traffic_summary
    GROUP BY VehicleType, ExitPoint
""").show()

+-----------+---------+---------+
|VehicleType|ExitPoint|TotalToll|
+-----------+---------+---------+
|        Car|    GateD|       50|
|      Truck|    GateC|      100|
|       Bike|    GateD|       30|
|        Bus|    GateA|       70|
|        Car|    GateC|       50|
+-----------+---------+---------+

